In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import pyhf
from pyhf.contrib.viz import brazil

In [2]:
MJData = pd.read_csv("MJData.csv")
MJ_bkg = MJData["MJ_bkg"]
MJ_errbkg = MJData["MJ_errbkg"]
MJ_data = MJData["MJ_data"]
MJ_higgs = MJData["MJ_higgs"]
MJ_axial = MJData["MJ_axial"]

toySignal = pd.read_csv("pt_miss_histogram_monojet_BMtoy.csv")["counts"]

In [3]:
toyData = MJ_data + toySignal

In [4]:
signalPar = np.linspace(1.106, 1.11, 100)

for i in signalPar:
    model1 = pyhf.simplemodels.uncorrelated_background(
        signal=list(i*toySignal), bkg=list(MJ_bkg), bkg_uncertainty=list(MJ_errbkg)
    ) #creamos el modelo

    observations1 = np.concatenate((toyData, model1.config.auxdata))

    poi_values1 = np.linspace(0.01, 5, 50)
    obs_limit1, exp_limits1, (scan, results) = pyhf.infer.intervals.upper_limits.upper_limit(
        observations1, model1, poi_values1, level=0.05, return_results=True
    )
    if abs(1 - obs_limit1) < 1e-3:
        print(abs(1 - obs_limit1), i)
        break
    
print(f"Upper limit (obs): μ = {obs_limit1:.4f}")
print(f"Upper limit (exp): μ = {exp_limits1[2]:.4f}")

In [ ]:
#p-value
pvalPar = np.linspace(1.02, 1.06, 100)

for i in pvalPar:
    model1 = pyhf.simplemodels.uncorrelated_background(
        signal=list(i*toySignal), bkg=list(MJ_bkg), bkg_uncertainty=list(MJ_errbkg)
    ) #creamos el modelo

    observations1 = np.concatenate((toyData, model1.config.auxdata))

    pvalue1 = pyhf.infer.hypotest(
        1, observations1, model1, return_expected_set=True, return_tail_probs = True
    )[1][0]
    if abs(0.05 - pvalue1) < 1e-3:
        print(abs(0.05 - pvalue1), i)
        break

print("p-value = ", pvalue1)

0.0006291270379414071 1.0426262626262626
p-value =  0.049370872962058596


as